<a href="https://colab.research.google.com/github/yejijang-analyst/ESAA/blob/main/note/24-1/5_1_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8. 텍스트 분석



- 텍스트 분류
- 감성 분석
- 텍스트 요약
- 텍스트 군집화와 유사도 측정


## 01 텍스트 분석 이해
중요: 피쳐벡터화

<텍스트 분석 수행 프로세스>
1. 텍스트 전처리
  - 클렌징,  단어 토큰화, 어근 추출, 텍스트 정규화

2. 피처 벡터화 / 추출 :  Word2Vec

3. ML 모델링


->  **텍스트 전처리**

- 클렌징
-  단어 토큰화
- 스톱워드 제거
- Stemming




## 클렌징 & 토큰화


In [1]:
from nltk import sent_tokenize
import nltk
nltk.download('punkt')

text_sample = 'The Matrix is everywhere its all around us, here even in this room. \
                You can see it out your window or on your television. \
                You feel it when you go to work, or go to church or pay your taxes.'
sentences = sent_tokenize(text = text_sample)

print(type(sentences), len(sentences))
print(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<class 'list'> 3
['The Matrix is everywhere its all around us, here even in this room.', 'You can see it out your window or on your television.', 'You feel it when you go to work, or go to church or pay your taxes.']


In [2]:
from nltk import word_tokenize

sentence = 'The Matrix is everywhere its all around us, here even in this room.'
words = word_tokenize(sentence)
print(type(words), len(words))
print(words)

<class 'list'> 15
['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.']


sent_tokenize와 word_tokenize를 조합

In [3]:
from nltk import word_tokenize, sent_tokenize

# 여러 개의 문장으로 된 입력 데이터를 문장별로 단어 토큰화하게 만드는 함수 생성
def tokenize_text(text):

    # 문장별로 분리 토큰
    sentences = sent_tokenize(text)

    # 분리된 문장별 단어 토큰화
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens

# 여러 문장에 대해 문장별 단어 토큰화 수행
word_tokens = tokenize_text(text_sample)
print(type(word_tokens), len(word_tokens))
print(word_tokens)

<class 'list'> 3
[['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.'], ['You', 'can', 'see', 'it', 'out', 'your', 'window', 'or', 'on', 'your', 'television', '.'], ['You', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


문맥적 의미가 무시

###스톱워드 제거


In [4]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
print('영어 stop words 개수 : ', len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english')[:20])

영어 stop words 개수 :  179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [6]:

import nltk

stopwords = nltk.corpus.stopwords.words('english')
all_tokens = []

for sentence in word_tokens :
    filtered_words = []
    for word in sentence :

        word = word.lower()

        if word not in stopwords :
            filtered_words.append(word)
    all_tokens.append(filtered_words)

print(all_tokens)

[['matrix', 'everywhere', 'around', 'us', ',', 'even', 'room', '.'], ['see', 'window', 'television', '.'], ['feel', 'go', 'work', ',', 'go', 'church', 'pay', 'taxes', '.']]


is, this 같은 스톱워드 제거

### Stemming 과 Lemmatization
-  단어의 원형을 찾기

**Lemmatization**이 Stemming보다 정교하며 의미론적인 기반에서 단어의 원형을 찾는다

- WordNetLemmatizer


**Stemming**은 원형 단어로 변환 시 일반적인 방법을 적용하거나 더 단순화된 방법을 적용

In [7]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()

print(stemmer.stem('working'), stemmer.stem('works'), stemmer.stem('worked'))
print(stemmer.stem('amusing'), stemmer.stem('amuses'), stemmer.stem('amused'))
print(stemmer.stem('happier'), stemmer.stem('happiest'))
print(stemmer.stem('fancier'), stemmer.stem('fanciest'))

work work work
amus amus amus
happy happiest
fant fanciest


- WordNetLemmatizer
- 정확한 원형단어추출을 위해 단어의 품사를 입력필요

In [8]:
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')

lemma = WordNetLemmatizer()
print(lemma.lemmatize('amusing', 'v'), lemma.lemmatize('amuses', 'v'), lemma.lemmatize('amused', 'v'))
print(lemma.lemmatize('happier', 'a'), lemma.lemmatize('happiest', 'a'))
print(lemma.lemmatize('fancier', 'a'), lemma.lemmatize('fanciest', 'a'))

[nltk_data] Downloading package wordnet to /root/nltk_data...


amuse amuse amuse
happy happy
fancy fancy


## 03 Bag of Words - BOW

- 단어의 문맥이나 순서를 무시하고 일괄적으로 단어에 대해 빈도 값을 부여해 피처 값 추출

문장 2개가 있을 때..
1. 문장1과 2에 있는 모든 단어에서 중복을 제거하고 각 단어를 칼럼 형태로 나열하고 각 단어에 고유의 인덱스를 부여

2. 개별 문장에서 해당 단어가 나타나는 횟수를 각 단어에 기재

단점
- 문맥 의미 반영 부족
- 희소 행렬 문제 (희소성, 희소행렬)
  - 대규모의 칼럼에서 구성된 행렬에서 대부분의 값이 0인 행렬

### BOW 피처 벡터화

1. 카운트 기반 벡터화
- 단어 횟수
- 카운트 높으면 중요한 단어라고 고려

2. TF-IDF (Term Frequency - Inverse Document Frequency) 기반 벡터화
- 자주 나타나는 단어에 높은 가중치, ㅗ든 문서에 자주 나타나는 단어에 대해서는 패널티 부여

### CountVectorizer, TfidfVectorizer


파라미터
- max_df : 너무 높은 빈도수를 가지는 단어 피처를 제외
- min_df : 너무 낮은 빈도수를 가지는 단어 피처를 제외
- max_features : 피처의 개수를 제한
- n_gram_range : tuple 형태로 범위 최솟값, 범위 최댓값을 지정
- analyzer : 피처 추출을 수행한 단위, 디폴트는 word
- token_pattern : 토큰화를 수행하는 정규 표현식 패턴 지정
-tokenizer : 토큰화를 별도의 커스텀 함수로 이용시 적용



**TfidfVectorizer**

파라미터 동일

### 희소행렬 :COO 형식
별도의 데이터 배열에 0이 아닌 데이터만 저장하고 데이터가 가리키는 행과 열의 위치를 별도의 배열로 저장

In [9]:
import numpy as np

dense = np.array([[3,0,1], [0,2,0]])

In [10]:
from scipy import sparse

# 0이 아닌 데이터 추출
data = np.array([3,1,2])

# 행 위치와 열 위치를 각각 배열로 생성
row_pos = np.array([0,0,1])
col_pos = np.array([0,2,1])

# sparse 패키지의 coo_matrix를 이용해 COO 형식으로 희소 행렬 생성
sparse_coo = sparse.coo_matrix((data, (row_pos, col_pos)))

In [11]:
sparse_coo.toarray()

array([[3, 0, 1],
       [0, 2, 0]])

### 희소행렬 - CSR 형식
COO 형식의 반복적인 위치 데이터를 사용해야하는 것 해결


In [12]:
from scipy import sparse

dense = np.array([[0,0,1,0,0,5],
                  [1,4,0,3,2,5],
                  [0,6,0,3,0,0],
                  [2,0,0,0,0,0],
                  [0,0,0,7,0,8],
                  [1,0,0,0,0,0]

])

# 0이 아닌 데이터 추출
data2 = np.array([1,5,1,4,3,2,5,6,3,2,7,8,1])

# 행 위치와 열 위치를 각각 array로 생성
row_pos = np.array([0,0,1,1,1,1,1,2,2,3,4,4,5])
col_pos = np.array([2,5,0,1,3,4,5,1,3,0,3,5,0])

# COO 형식으로 변환
sparse_coo = sparse.coo_matrix((data2, (row_pos, col_pos)))

# 행 위치 배열의 고유한 값의 시작 위치 인덱스를 배열로 생성
row_pos_ind = np.array([0,2,7,9,10,12,13])

# CSR 형식으로 변환
sparse_csr = sparse.csr_matrix((data2, col_pos, row_pos_ind))

print('COO 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_coo.toarray())
print('CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_csr.toarray())

COO 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]
CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]


In [13]:
dense3= np.array([[0,0,1,0,0,5],
                  [1,4,0,3,2,5],
                  [0,6,0,3,0,0],
                  [2,0,0,0,0,0],
                  [0,0,0,7,0,8],
                  [1,0,0,0,0,0]

])

coo = sparse.coo_matrix(dense3)
csr = sparse.csr_matrix(dense3)